In [1]:
from sqlalchemy import create_engine
import pandas as pd

## Data

In [2]:
test_data = {
    'cookies':[
        ['cookie_id', 'cookie_name', 'cookie_recipe_url', 'cookie_sku', 'quantity', 'unit_cost'],
        [1,'dark chocolate chip','http://some.aweso.me/cookie/recipe_dark.html','CC02',1,0.75],
        [2,'peanut butter','http://some.aweso.me/cookie/peanut.html','PB01', 24,0.25],
        [3,'oatmeal raisin','http://some.okay.me/cookie/raisin.html','EWW01',100,1.00]
    ],
    'users':[
        ['email_address', 'password', 'phone', 'user_id', 'username'],
        ['mon@cookie.com', 'password', '111-111-1111', 1, 'cookiemon'],
        ['cakeeater@cake.com', 'password', '222-222-2222', 2, 'cakeeater'],
        ['guy@pie.com', 'password', '333-333-3333', 3, 'pieguy']
    ],
    'line_items' : [
        ['cookie_id','extended_cost','order_id','quantity'],
        [1,1.0,'wlk001',2],
        [3,3.0,'wlk001',12],
        [1,12.0,'ol001',24],
        [2,6.0,'ol001',6]
    ],
    'orders' : [
        ['order_id','user_id','shipped'],
        ['wlk001', 1,True], 
        ['ol001', 2,False]
    ]
}

In [3]:
tables = """\
DROP TABLE IF EXISTS cookies;
CREATE TABLE cookies (
	cookie_id INTEGER,
	cookie_name VARCHAR(50),
	cookie_recipe_url VARCHAR(255),
	cookie_sku VARCHAR(55),
	quantity INTEGER,
	unit_cost NUMERIC(12, 2)
);

DROP TABLE IF EXISTS users;
CREATE TABLE users (
	user_id INTEGER,
	customer_number INTEGER,
	username VARCHAR(15) NOT NULL,
	email_address VARCHAR(255) NOT NULL,
	phone VARCHAR(20) NOT NULL,
	password VARCHAR(25) NOT NULL,
	UNIQUE (username)
);

DROP TABLE IF EXISTS line_items;
CREATE TABLE line_items (
	line_items_id INTEGER,
	order_id VARCHAR(20),
	cookie_id INTEGER,
	quantity INTEGER,
	extended_cost NUMERIC(12, 2)
);

DROP TABLE IF EXISTS orders;
CREATE TABLE orders (
	order_id VARCHAR(20),
	user_id INTEGER,
	shipped BOOLEAN
);
"""

In [4]:
get_orders_by_customer_SQL = """\
WITH get_orders AS (
    SELECT
        t1.order_id,
        t2.username,
        t2.phone
    FROM orders as t1
    INNER JOIN users as t2
    ON t1.user_id = t2.user_id
    WHERE t2.username = '@{username}'  
    AND t1.shipped in (@{shipped})
    ),
    
get_ordersdetails AS (
    SELECT 
        t1.*,
        t3.cookie_name,
        t2.quantity,
        t2.extended_cost 
    FROM get_orders as t1
    INNER JOIN line_items as t2
    on t1.order_id = t2.order_id
    INNER JOIN cookies as t3
    on t2.cookie_id = t3.cookie_id
    )
    
SELECT * 
FROM get_orders@{details};
""" 

In [5]:
def get_orders_by_customer(username,shipped='False,True',details=''):
    sql = get_orders_by_customer_SQL.replace('@{username}',username)
    sql = sql.replace('@{shipped}',shipped)
    sql = sql.replace('@{details}',details)
    return sql   

## Unit Tests

In [6]:
import unittest
from decimal import Decimal

class TestApp(unittest.TestCase):
    
    cookie_orders = [(u'wlk001', u'cookiemon', u'111-111-1111')]
    cookie_details = [
        (u'wlk001', u'cookiemon', u'111-111-1111',
            u'dark chocolate chip', 2, Decimal('1.00')),
        (u'wlk001', u'cookiemon', u'111-111-1111',
            u'oatmeal raisin', 12, Decimal('3.00'))]
    
    @classmethod
    def setUpClass(cls):      
        # Connect to the database and create the schema within a transaction
        cls.conn_string='sqlite:///:memory:'
        cls.engine = create_engine(cls.conn_string)
        cls.connection = cls.engine.connect()
        cls.transaction = cls.connection.begin()
        #Create the tables
        for statement in tables.split(';')[:-1]:
            cls.connection.execute(statement)
        #Populate the tables
        for table,values in test_data.items():
            df = pd.DataFrame(values[1:],columns=values[0])
            df.to_sql(table,cls.connection,
                      if_exists='replace',index=False,schema=None)
       
    @classmethod
    def tearDownClass(cls):
        # Roll back the top level transaction and disconnect from the database
        cls.transaction.rollback()
        cls.connection.close()
        cls.engine.dispose()
        
    def test_orders_by_customer_blank(self):
        results = self.connection.execute(get_orders_by_customer('')).fetchall()
        self.assertEqual(results, [])

In [7]:
unittest.main(argv=[''], verbosity=2, exit=False);


test_orders_by_customer_blank (__main__.TestApp) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.053s

OK
